# Controller for the Servo and Lever Box Proof-of-concept.

This script defines a simple interactive MDRNN application for a musical interface with one real-valued input, a sound processing system, and one physical output.

In [2]:
import serial
import time
import struct
import socket
from threading import Thread
import musical_mdn
import numpy as np
import pandas as pd
import tensorflow as tf
musical_mdn.MODEL_DIR = "./"

# Details for OSC output
ADDRESS = "localhost"
PORT = 5000
INT_FLOAT_DGRAM_LEN = 4
STRING_DGRAM_PAD = 4

# Socket for OSC output.
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.setblocking(0)

# Serial for input and output via a USB-connected microcontroller.
ser = serial.Serial('/dev/cu.usbmodem1461', 9600)

## Functions for OSC Connection.

def send_sound_command(osc_datagram):
    """Send OSC message via UDP."""
    sock.sendto(osc_datagram, (ADDRESS, PORT))

def pad_dgram_four_bytes(dgram):
    """Pad a datagram up to a multiple of 4 bytes."""
    return (dgram + (b'\x00' * (4 - len(dgram) % 4)))

def touch_message_datagram(pos = 0.0):
    """Construct an osc message with address /touch and one float."""
    dgram = b''
    dgram += pad_dgram_four_bytes("/touch".encode('utf-8'))
    dgram += pad_dgram_four_bytes(",f")
    dgram += struct.pack('>f', pos)
    return dgram

# Functions for sending and receving from levers.

def command_servo(input=128):
    """Send a command to the servo. Input is between 0, 255"""
    ser.write(input)
    
def read_lever():
    """Read a single byte from the lever and return as integer."""
    return ord(ser.read(1))

# Functions for playing back level sounds and moving servo

def move_and_play_sound(loc):
    """Move the servo and play a sound in response to a byte input."""
    command_servo(loc)
    send_sound_command(touch_message_datagram(loc / 255.0))
    
def clear_and_play_serial_input(movement = True):
    """Clears the serial buffer of input and plays it all."""
    while ser.in_waiting > 0:
        loc = read_lever()
        send_sound_command(touch_message_datagram(loc / 255.0))
        if movement:
            command_servo(loc)

## MDN      
net = musical_mdn.TinyJamNet2D(mode = musical_mdn.NET_MODE_RUN, n_hidden_units = 128, n_mixtures = 10, batch_size = 1, sequence_length = 1)

SerialException: [Errno 2] could not open port /dev/cu.usbmodem1461: [Errno 2] No such file or directory: '/dev/cu.usbmodem1461'

# Perform from Model

This code sets up a performance demonstration with the model.

- Human response run loop is set up to connect the input level to the sound processor
- Human run loop is threaded
- A 30 second performance is sampled from the network and performed by the script in the main thread.
- After this performance all run loops are stopped.


In [ ]:
listening_as_well = False
first_touch = np.array([(0.01 + (np.random.rand()-0.5)*0.005), (np.random.rand()-0.5)*20])

def threaded_function():
    ## Reacts to serial with sound, not movement
    while keep_reacting_to_lever:
        while ser.in_waiting > 0:
            loc = read_lever()
            send_sound_command(touch_message_datagram(loc / 255.0))

# Start a thread for listening
if listening_as_well:
    keep_reacting_to_lever = True
    thread = Thread(target = threaded_function)
    thread.start()
            
# Start TF Predictions
with tf.Session() as sess:
    # Get network ready to run
    net.prepare_model_for_running(sess)
    last_touch = first_touch
    time_total = 0
    count = 0
    
    print("Performing a 30s performance")
    while time_total < 30.0:
        # generate some output and schedule sounds and movement.
        last_touch = net.generate_touch(last_touch, sess)
        output = last_touch.reshape((2,))
        # convert to dt, byte format
        dt = output[0]
        pos = output[1]
        pos = int((pos + 10) * 255 / 20.0)
        pos = min(max(pos, 0), 255)
        dt = max(dt, 0)
        #print(dt)
        time.sleep(dt)
        move_and_play_sound(pos)
        time_total += dt
        count += 1
    print("Done, time was", time_total, "with", count, "moves.")
    keep_reacting_to_lever = False
    
if listening_as_well:
    thread.join()
    print("thread finished...exiting")

# Alternative Run Loop

- No prediction 
- Just ties output lever movement and sound to input lever. 
- Just for demonstration and debugging.

In [ ]:
def run_loop():
    while ser.in_waiting > 0:
        loc = read_lever() # read from the input lever
        send_sound_command(touch_message_datagram(loc / 255.0)) # send a sound command
        command_servo(loc) # output command to servo

print("Now running reaction loop...")
try:
    while True:
        run_loop()
except KeyboardInterrupt:
    pass
finally:
    m.disconnect()
    print("\nReaction Loop Finished")